In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import copy
import random
import socket
import sys
import time
import unicodedata
import urllib
from subprocess import call

from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.keys import Keys


#검색할 항목 입력
term = "로고"  
#저장 경로인대 안먹히는듯
destination = "C:/Users/shfks/OneDrive/Desktop/pinterest"
#갱신시간을 위해 딜레이를 줌
def randdelay(a, b):
    time.sleep(random.uniform(a, b))

#유니코드 정규화 
def u_to_s(uni):
    return unicodedata.normalize('NFKD', uni).encode('ascii', 'ignore')


class PinterestHelper(object):

    def __init__(self, login_name, pw, download=True):
        self.login_name = login_name 
        self.pw = pw
        self.download = download
        # self.browser = webdriver.Firefox()
        self.browser = webdriver.Chrome('C:/chromedriver/chromedriver.exe')
        self.browser.get("https://www.pinterest.com")
        self.browser.implicitly_wait(5)
     
    def login(self):
        email_elem = self.browser.find_element_by_name('id')
        email_elem.send_keys(self.login_name)
        password_elem = self.browser.find_element_by_name('password')
        password_elem.send_keys(self.pw)
        password_elem.send_keys(Keys.RETURN)
        randdelay(2, 4)
     

    def runme(self, url, threshold=50): # threshold = 스크롤바 횟수
        final_results = []
        previmages = []
        tries = 0
        try:
            self.browser.get(url)
            while threshold > 0:
                try:
                    results = []
                    images = self.browser.find_elements_by_tag_name("img")
                    if images == previmages: # 이미지 중복 체크
                        tries += 1
                    else:
                        tries = 0
                    if tries > 20:
                        return final_results
                    for i in images:
                        src = i.get_attribute("src") # src = 이미지 경로.jpg  
                        if src:
                            if src.find("/236x/") != -1 or src.find("/474x/") != 1:
                                print(src)
                                src = src.replace("/236x/", "/736x/")
                                src = src.replace("/474x/", "/736x/")
                                results.append(u_to_s(src))

                    previmages = copy.copy(images)
                    final_results = list(set(final_results + results))
                    dummy = self.browser.find_element_by_tag_name('a')
                    dummy.send_keys(Keys.PAGE_DOWN) #스크롤바 내림
                    randdelay(0, 1)
                    threshold -= 1
                except StaleElementReferenceException:
                    threshold -= 1
        except (socket.error, socket.timeout):
            pass
        return final_results

    def close(self):
        #브라우저 닫기
        self.browser.close()


def main():
    #아이디 입력
    PINTEREST_USERNAME = "sensemint_@naver.com" 
    #비밀번호 입력
    PINTEREST_PASSWORD = "poster99"
   # sys.argv는 프로그램을 실행할 때 입력된 값을 읽어 들일 수 있는 파이썬 라이브러리이다.
   #sys.argv[0]는 입력받은 값 중에서 파이썬 프로그램 이름인 .py이므로 필요 없는 값이다.
    if len(sys.argv) > 1:
         trem = sys.argv[1] 
    else:     
        exit()
    ph = PinterestHelper(PINTEREST_USERNAME, PINTEREST_PASSWORD) #초기화 밎 크롬 연결
    sleep(3)
    ph.login() #로그인
    sleep(3.2)
    is_url = urllib.parse.urlparse(term)
    #is_url.scheme  = 'https'  , is_url.netloc = www.pinterest.co.kr
    if is_url.scheme and is_url.netloc:
        images = ph.runme(term)
    else:
        images = ph.runme('http://pinterest.com/search/pins/?q=' + urllib.parse.quote(term))
    print(images)
    ph.close()
    with open(term.replace(" ", "") + "_pins.txt", "w") as file:
        file.write('\n'.join([i.decode('UTF-8') for i in images]))
    if len(sys.argv) > 2:
           destination = sys.argv[2]  
    else:
        destination = "./" + term.replace(" ", "")

    print(term, destination)
    #서브프로세스. shell결과를 파일에 저장
    call('aria2c -i ./{}_pins.txt -d {} --continue --auto-file-renaming false'.format(term.replace(" ", ""),
                                                                                      destination),
         shell=True)


if __name__ == '__main__':
    main()

https://s.pinimg.com/images/user/default_75.png
https://i.pinimg.com/236x/99/00/81/9900816d10f5ef762e631f37647e77eb.jpg
https://i.pinimg.com/236x/54/73/27/54732781088a26975a99cdc7fa62fea8.jpg
https://i.pinimg.com/236x/49/0e/17/490e17c6df34043fd9ef4749d5f87ef7.jpg
https://i.pinimg.com/236x/6d/20/b2/6d20b28ad0d36ff182ab4cf88e0ac4ef.jpg
https://i.pinimg.com/236x/1f/61/9b/1f619b3178fc5e3b1e8f2bb4368db185.jpg
https://i.pinimg.com/236x/95/70/cf/9570cfca3a83f61f35547b834e31db5a.jpg
https://i.pinimg.com/236x/74/66/e4/7466e497721623fb6d2e004fd72e019d.jpg
https://i.pinimg.com/236x/ee/00/ad/ee00adc620e7dc6dc924ccb1903aac13.jpg
https://i.pinimg.com/236x/46/a9/da/46a9daec68dd28e2f9df81b1fda13341.jpg
https://i.pinimg.com/236x/d3/fe/b5/d3feb5a4ab0c8b46a0fda9d0788073f3.jpg
https://i.pinimg.com/236x/e5/2b/64/e52b644301888c7d0c92a7cf5af16dad.jpg
https://i.pinimg.com/236x/b0/d4/9d/b0d49df5241baa70e668fb2432893691.jpg
https://i.pinimg.com/236x/cb/35/3d/cb353d9ea2765515909a8ab97fa82792.jpg
https://i.pinimg

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=80.0.3987.163)
